## Question 5
You are a life actuary of an insurance company. Today on 13 March 2022, a potential
client, Alice, is looking to sign a permanent life insurance contract with your company.
Alice, born on 28 October 1983, is a smoker with the health condition standard for her
cohort with the same age, gender, and smoker status. You decide to apply the 2015
Valuation Basic Table (VBT) (www.soa.org/files/research/exp-study/2015-vbt-smokerdistinct-alb-anb.xlsx) for Alice’s mortality evaluation with the Age Nearest Birthday
(ANB) method for her age calculation. Your company applies the valuation interest rate
of 3.5% for premium pricing:

a) Calculate Alice’s annal unconditional survival rates $_np_x$ from now to age 42. [3
marks]

b) Calculate Alice’s life expectancy in years (show one decimal place). [2 marks]

c) Calculate the probability that Alice dies exactly between 6 and 9 years from now
(show five decimal places). [2 marks]

d) Calculate the minimum annual premium rate (premium as a fraction of death
benefit) that your company should charge for Alice’s cohort (show five decimal
places). [3 marks]

e) What technologies can you apply to enable a closer prediction on the company’s
cash flow linked to this line of business of permanent life insurance? [answers
should be no longer than 200 words] [4 marks]

In [63]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [79]:
df = pd.read_excel('./data/2015-vbt-smoker-distinct-alb-anb.xlsx', header=2, sheet_name='2015 FSM ANB')
df.set_index = df['Iss. Age']
df

,Iss. Age,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,Ult.,Att. Age
0,18,0.27,0.27,0.27,0.27,0.27,0.29,0.33,0.34,0.34,...,0.73,0.86,1.03,1.17,1.29,1.40,1.49,1.58,1.67,43
1,19,0.27,0.27,0.26,0.26,0.27,0.31,0.33,0.33,0.33,...,0.86,1.01,1.16,1.29,1.40,1.49,1.58,1.67,1.76,44
2,20,0.24,0.23,0.24,0.25,0.29,0.31,0.32,0.32,0.34,...,1.00,1.14,1.27,1.40,1.49,1.58,1.67,1.76,1.87,45
3,21,0.19,0.21,0.23,0.26,0.29,0.30,0.31,0.33,0.37,...,1.13,1.26,1.38,1.49,1.58,1.67,1.76,1.87,2.05,46
4,22,0.17,0.20,0.24,0.27,0.28,0.30,0.32,0.35,0.39,...,1.24,1.36,1.48,1.58,1.67,1.76,1.87,2.05,2.25,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,91,41.15,70.04,116.51,194.74,207.13,222.69,238.98,255.83,272.73,...,462.77,476.86,488.00,495.83,500.00,500.00,500.00,500.00,500.00,116
74,92,51.27,101.54,194.74,207.13,222.69,238.98,255.83,272.73,289.19,...,476.86,488.00,495.83,500.00,500.00,500.00,500.00,500.00,500.00,117
75,93,66.14,140.43,207.13,222.69,238.98,255.83,272.73,289.19,313.30,...,488.00,495.83,500.00,500.00,500.00,500.00,500.00,500.00,500.00,118
76,94,86.12,159.72,222.69,238.98,255.83,272.73,289.19,313.30,337.47,...,495.83,500.00,500.00,500.00,500.00,500.00,500.00,500.00,500.00,119


In [80]:
alice_bday=date(1983, 10, 28)
alice_age = (date.today().year - alice_bday.year)
alice_age

39

In [85]:
series = df.loc[df['Iss. Age'] == alice_age]
series

,Iss. Age,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,Ult.,Att. Age
21,39,0.39,0.57,0.77,0.94,1.08,1.21,1.34,1.7,1.96,...,5.34,5.93,6.58,7.31,8.12,9.01,10.01,11.09,12.28,64


### Part 1
Calculate Alice’s annal unconditional survival rates $_np_x$ from now to age 42. [3 marks]

In [103]:
np_series = df.loc[df['Iss. Age'] == alice_age].drop(columns=['Iss. Age', 'Ult.', 'Att. Age']).to_numpy()[0]
np_series[0:42-alice_age]

array([0.39, 0.57, 0.77])

### Part 2 (fix)
Calculate Alice’s life expectancy in years (show one decimal place)

In [119]:
i = 1
for count, idx in enumerate(np_series):
    i *= idx
    if i > 100:
        print("Age: ", count+alice_age)
        break

Age:  52


### Part 3
Calculate the probability that Alice dies exactly between 6 and 9 years from now (show five decimal places)